## TrainTestSplit

Create a train-test-split for the datasets found in the pipeline

In [130]:
# imports
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
import numpy as np
import os
import pandas as pd

In [131]:
# makedirs if not exist
os.makedirs("../../etl/data/intermediate/TrainTestSplit", exist_ok=False)

FileExistsError: [Errno 17] File exists: '../../etl/data/intermediate/TrainTestSplit'

In [132]:
# parameters
FULL_DATA_PATH="../../etl/data/raw/01_extract.csv"
TRAIN_DATA_PATH="../../etl/data/intermediate/TrainTestSplit/01_train.csv"
TEST_DATA_PATH="../../etl/data/intermediate/TrainTestSplit/01_test.csv"
VAL_DATA_PATH="../../etl/data/intermediate/TrainTestSplit/01_val.csv"
RANDOM_STATE=42

In [243]:
df = pd.read_csv(FULL_DATA_PATH)

In [244]:
# group by sentences
df

,doc_id,verb_form,verb_form_start,verb_form_end,verb_lemma,arg1,arg1_start,arg1_end,arg1_pos,arg1_head,...,arg2,arg2_start,arg2_end,arg2_pos,arg2_head,arg2_head_start,arg2_head_end,rel_type,pred_serial,full_sentence_text
0,4bc8c13ddaa028e64a34ce08397157b846fb4de3ad26e34759aa57fdbeb50bcc,abgestraft,26,36,abstrafen,Alexis Tsipras,5,19,N,Alexis,...,Dass Alexis Tsipras jetzt abgestraft wurde,0,43,$.,.,143,144,neutral,"Predicate(type='neutral', args=(Head(sentence=6, token=1), Head(sentence=6, token=-1)), strength=0, verb=4)","Dass Alexis Tsipras jetzt abgestraft wurde , hat viel mit der angestauten Unzufriedenheit über die langen Jahre des Sparens und Darbens zu tun ."
1,4bc8c13ddaa028e64a34ce08397157b846fb4de3ad26e34759aa57fdbeb50bcc,enttäuschen,140,151,enttäuschen,die neue Regierung,75,93,N,Regierung,...,die Hoffnungen auf einen spürbaren Aufschwung,94,139,N,Hoffnungen,98,108,neutral,"Predicate(type='neutral', args=(Head(sentence=18, token=13), Head(sentence=18, token=15)), strength=0, verb=20)","Wenn die Kreditgeber Athen nicht zusätzlichen Spielraum öffnen , wird auch die neue Regierung die Hoffnungen auf einen spürbaren Aufschwung enttäuschen ."
2,4bc8c13ddaa028e64a34ce08397157b846fb4de3ad26e34759aa57fdbeb50bcc,beenden,119,126,beenden,Will er dem Land etwas Gutes tun,0,33,$.,.,...,die politische Polarisierung beenden,90,127,N,Polarisierung,105,118,neutral,"Predicate(type='neutral', args=(Head(sentence=10, token=-1), Head(sentence=10, token=20)), strength=0, verb=21)","Will er dem Land etwas Gutes tun , dann sollte er nicht nur Steuern senken , sondern auch die politische Polarisierung beenden , die das Klima in Griechenland zuletzt so vergiftet hat ."
3,290f3971010f6d9385e896208f328948f5fb3f9bc0caeb9508b4f1acc63a35ac,akzeptieren,69,80,akzeptieren,Pajtim Kasami,0,14,N,Pajtim,...,die Kurzarbeit nun doch,81,105,N,Kurzarbeit,85,95,pro,"Predicate(type='pro', args=(Head(sentence=12, token=0), Head(sentence=12, token=13)), strength=0, verb=11)","Pajtim Kasami , Ermir Lenjani , Birama Ndoye und Mickael Facchinetti akzeptieren die Kurzarbeit nun doch , weshalb Sion-Präsident Christian Constantin die Entlassungen zurückzieht ."
4,290f3971010f6d9385e896208f328948f5fb3f9bc0caeb9508b4f1acc63a35ac,entliess,30,38,entlassen,der FC Sion,39,50,N,FC,...,Fussball Neun Spieler,8,29,N,Fussball,8,16,neutral,"Predicate(type='neutral', args=(Head(sentence=10, token=8), Head(sentence=10, token=3)), strength=0, verb=6)","( dpa ) Fussball Neun Spieler entliess der FC Sion Mitte März , als die Corona-Krise den Schweizer Fussball traf ."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832168,9701f0c776430a365f0619836f34658459be788b9a6646e86c252c477ef565a4,äussert,5,12,äussern,Frau A,13,19,N,Frau,...,massive Vorwürfe gegen ihre Vorgesetzten,20,60,N,Vorwürfe,28,36,neutral,"Predicate(type='neutral', args=(Head(sentence=13, token=2), Head(sentence=13, token=5)), strength=0, verb=1)",Dann äussert Frau A massive Vorwürfe gegen ihre Vorgesetzten .
832169,9701f0c776430a365f0619836f34658459be788b9a6646e86c252c477ef565a4,stelle,184,190,stellen,das Kommando,171,183,N,Kommando,...,einen Antrag auf ihren Ausschluss,191,224,N,Antrag,197,203,neutral,"Predicate(type='neutral', args=(Head(sentence=28, token=31), Head(sentence=28, token=34)), strength=0, verb=32)","Am 5. Dezember stellt der Chef der Abteilung Milizfeuerwehr und Zivilschutz Frau A vor die Wahl : Entweder trete sie per Ende des Jahres aus der Milizfeuerwehr aus , oder das Kommando stelle einen Antrag auf ihren Ausschluss ."
832170,d0fc434ce0021b0dff7b52157d352daaff8d1a65f4640a744651af0e992064bf,ausgewiesen,77,88,ausweisen,Der Konzern,0,11,N,Konzern,...,eine Liquidität,32,47,N,Liquidität,37,47,con,"Predicate(type='con', args=(Head(sentence=19, token=1), Head(sentence=19, token=7)), strength=0, verb=13)",Der Konzern hatte im April noch eine Liquidität von gut vier Milliarden Euro ausgewiesen .
832171,d0fc434ce0021b0dff7b52157d352daaff8d1a65f4640a744651af0e992064bf,belebt

**Clean dataset**

As seen below we must remove the argument heads make sure we don't have any "." as arguments for the neutrals.

In [246]:
mask = ~((df["arg1_head"] == ".")
|(df["arg2_head"] == ".")
|(df["arg2"] == ".")
|(df["arg2"] == "."))

# print(mask)

df = df[mask]

**Balance the dataset**

Make sure that each class is represented equally.

In [247]:
df = df.groupby('rel_type')
df = df.apply(lambda x: x.sample(df.size().min()).reset_index(drop=True))

df.rel_type.value_counts()

con        89673
neutral    89673
pro        89673
Name: rel_type, dtype: int64

**How big is the problem with multi-PAS per sentence?**

Which should not be split accross the dataset splits.

In [248]:
df_occ = df.copy(deep=True)
df_occ = df_occ.merge(df_occ.groupby(["doc_id", "full_sentence_text"]).size().reset_index(name="counts"), on=["doc_id", "full_sentence_text"])
df_occ_freq = df_occ.groupby(["counts"]).size()
df_occ_freq

counts
1    258817
2      9660
3       522
4        20
dtype: int64

### Complex sentences analysis

**How do the multi-PAS sentences look**?

Complexity and sentence analysis.

In [249]:
pd.set_option('display.max_colwidth', None)
s1 = df_occ[df_occ["counts"] >= 2] \
    .sort_values(["full_sentence_text"]) \
    .sample(n=1)
s1_val = s1["full_sentence_text"].to_list()[0]
df_occ[df_occ["full_sentence_text"] == s1_val]

,doc_id,verb_form,verb_form_start,verb_form_end,verb_lemma,arg1,arg1_start,arg1_end,arg1_pos,arg1_head,...,arg2_start,arg2_end,arg2_pos,arg2_head,arg2_head_start,arg2_head_end,rel_type,pred_serial,full_sentence_text,counts
199001,716014e4587e60516a67bdf44a0e64216663e707fa6835fbe543a8b57a8e966f,hält,33,37,halten,Putin,43,48,N,Putin,...,58,64,N,Bedarf,58,64,pro,"Predicate(type='pro', args=(Head(sentence=67, token=7), Head(sentence=67, token=10)), strength=0, verb=5)","Ans Minsker Abkommen zur Ukraine hält sich Putin nur nach Bedarf , und das Budapester Memorandum , worin Russland 1994 Souveränität und Grenzen der Ukraine anerkannt hatte , ist heute Makulatur .",2
199002,716014e4587e60516a67bdf44a0e64216663e707fa6835fbe543a8b57a8e966f,anerkannt,156,165,anerkennen,Russland 1994,105,118,N,Russland,...,119,155,N,Souveränität,119,131,pro,"Predicate(type='pro', args=(Head(sentence=67, token=18), Head(sentence=67, token=20)), strength=0, verb=25)","Ans Minsker Abkommen zur Ukraine hält sich Putin nur nach Bedarf , und das Budapester Memorandum , worin Russland 1994 Souveränität und Grenzen der Ukraine anerkannt hatte , ist heute Makulatur .",2


**How big is the problem of -1 in first span or a "." in the data?**

The -1 does not exist. No problem currently.

The "." is quite prevalent and strangely only occurrs in the neutral case. That's why a cleaning step was inroduced above (see solution).

In [251]:
df_dirty = df.copy(deep=True)
df_dirty[
(df_dirty["arg1_head"] == ".")
|(df_dirty["arg2_head"] == ".")
|(df_dirty["arg2"] == ".")
|(df_dirty["arg2"] == ".")
#(df_dirty["arg1_start"] == -1)
#|(df_dirty["arg2_start"] == -1)
#|(df_dirty["arg2_head_start"] == -1)
#|(df_dirty["arg2_head_start"] == -1)
].rel_type.value_counts()

Series([], Name: rel_type, dtype: int64)

**Option 1: Train-test-splitting**

Only problem: We may have sentences within the same documents with multiple PAS that are split accross the sets.

In [252]:
train, test_val = train_test_split(df, test_size=0.3, stratify=df["rel_type"], random_state=RANDOM_STATE)
test, val = train_test_split(test_val, test_size=0.5, stratify=test_val["rel_type"],random_state=RANDOM_STATE)

print("VALUE COUNTS: train")
print(train.rel_type.value_counts())
train.to_csv(TRAIN_DATA_PATH, index=False)

print("VALUE COUNTS: test")
print(test.rel_type.value_counts())
test.to_csv(TEST_DATA_PATH, index=False)

print("VALUE COUNTS: val")
print(val.rel_type.value_counts())
val.to_csv(VAL_DATA_PATH, index=False)


VALUE COUNTS: train
neutral    62771
pro        62771
con        62771
Name: rel_type, dtype: int64
VALUE COUNTS: test
neutral    13451
pro        13451
con        13451
Name: rel_type, dtype: int64
VALUE COUNTS: val
pro        13451
con        13451
neutral    13451
Name: rel_type, dtype: int64


**Option 2: Train-test-splitting**

With respecting group distribution

In [253]:
# preserve groups between sentences (in this case doc_id is safe enough)
splitter = GroupShuffleSplit(test_size=.30, n_splits=1, random_state=RANDOM_STATE)
split = splitter.split(df, groups=df['doc_id'])
train_inds, test_val_inds = next(split)

train = df.iloc[train_inds]
test_val = df.iloc[test_val_inds]

splitter = GroupShuffleSplit(test_size=.5, n_splits=1, random_state=RANDOM_STATE)
split = splitter.split(test_val, groups=test_val['doc_id'])
test_inds, val_inds = next(split)

test = test_val.iloc[test_inds]
val = test_val.iloc[val_inds]

In [254]:
print("VALUE COUNTS: train")
print(train.rel_type.value_counts())
train.to_csv(TRAIN_DATA_PATH, index=False)

print("VALUE COUNTS: test")
print(test.rel_type.value_counts())
test.to_csv(TEST_DATA_PATH, index=False)

print("VALUE COUNTS: val")
print(val.rel_type.value_counts())
val.to_csv(VAL_DATA_PATH, index=False)

VALUE COUNTS: train
pro        62827
con        62695
neutral    62539
Name: rel_type, dtype: int64
VALUE COUNTS: test
con        13637
neutral    13596
pro        13395
Name: rel_type, dtype: int64
VALUE COUNTS: val
neutral    13538
pro        13451
con        13341
Name: rel_type, dtype: int64


In [67]:
# a small test to see whether option no 2 achieves our goals.

X = np.ones(shape=(10, 2))
y = np.ones(shape=(10, 1))
groups = np.array([1, 1, 2, 2, 2, 3, 3, 3, 8, 8])
print(groups.shape)

gss = GroupShuffleSplit(n_splits=1, train_size=.8, random_state=42)
gss.get_n_splits()

for train_idx, test_idx in gss.split(X, y, groups):
    print("TRAIN:", [groups[i] for i in train_idx], "TEST:", [groups[i] for i in test_idx])

(10,)
TRAIN: [1, 1, 3, 3, 3, 8, 8] TEST: [2, 2, 2]
